### Notebook zur Berechnung des Fuzzy-Inferenz-Schemas


In diesem Notebook kann das Fuzzy-Inferenz-Schema aus der Arbeit berechnet werden und anschließend mit einer Defuzzifizierungsmethode in einen scharfen Wert umgewandelt werden. Für mehr Infos zu den verwendeten Methoden siehe $\href{https://github.com/jdvelasq/fuzzy-expert/tree/main}{hier}$.


##### Importiere benötigte Pakete

In [ ]:
from fuzzy_expert.variable import FuzzyVariable
from fuzzy_expert.rule import FuzzyRule
from fuzzy_expert.inference import DecompositionalInference

### Input

#### Eingangswerte:

In [ ]:
age = 38
time = 3.5
P_kat1 = 2.5
P_kat2 = 4.3
P_kat3 = 8.5

#### Verwendete Operatoren für das Inferenz-Schema und Defuzzifizierung

'and_operator': Operator $f$ für Durchschnittsbildung, möglich sind z.B.:

"min": $f(a,b) = \min(a,b)$
<br>
"prod": $f(a,b) = a\cdot b$
<br>
"bunded_prod" =  $f(a,b) = \max(0, a + b - 1)$
<br>
<br>
'or_operator': Operator $f$ für Vereinigungsbildung, möglich sind z.B.:

"max": $f(a,b) = \max(a,b)$
<br>
"prob_or": $f(a,b) = a + b - a \cdot b$
<br>
"bunded_sum":  $f(a,b) = \min(1, a + b)$
<br>
<br>
'implication_operator': Operator $f$ für Fuzzy-Implikation, möglich sind z.B.:

"Rc": $f(a,b) = \min(a,b)$
<br>
"Ra": $f(a,b) = \min(1, 1 - a + b)$
<br>
"Rg":  $f(a,b) = \begin{cases}
1, & a\leq b \\
b, & \text{sonst}
\end{cases}$
<br>
<br>
'defuzzification_operator': Defuzzifizierungsmethode, möglich sind z.B.:

"cog": Center of gravity
<br>
"mom": Mean of maxima


In [ ]:
and_operator = "min"
or_operator = "max"
implication_operator = "Rc"
defuzzification_operator = "cog"

#### Definition der Fuzzy-Mengen

'age': Alter des Kunden
<br>
'time': Zeit seit Vertragsbeginn
<br>
'P_kat1': Performance Kategorie 1
<br>
'P_kat2': Performance Kategorie 2
<br>
'prob': Wechselwahrscheinlichkeit von Kategorie 1 nach Kategorie 2

In [ ]:
variables = {
    "age": FuzzyVariable(
        universe_range=(18, 100),
        terms={
            "jung": ('trapmf', 18, 18, 35, 40),
            "mittel": ('trapmf', 35, 40, 60, 67),
            "alt": ('trapmf', 60, 67, 100, 100),
        }
    ),
    "time": FuzzyVariable(
        universe_range=(0, 40),
        terms={
            "kurz": ('trapmf', 0, 0, 3, 5),
            "mittel": ('trapmf', 3, 5, 7, 9),
            "lang": ('trapmf', 7, 9, 40, 40),
        }
    ),
    "P_kat1": FuzzyVariable(
        universe_range=(-5, 20),
        terms={
            "nicht attraktiv": ('trapmf', -5, -5, 0, 2),
            "attraktiv": ('trapmf', 0, 2, 4, 6),
            "sehr attraktiv": ('trapmf', 4, 6, 20, 20),
        }
    ),
    "P_kat2": FuzzyVariable(
        universe_range=(-5, 25),
        terms={
            "nicht attraktiv": ('trapmf', -5, -5, 0, 5),
            "attraktiv": ('trapmf', 0, 5, 7, 12),
            "sehr attraktiv": ('trapmf', 7, 12, 25, 25),
        }
    ),
    "P_kat3": FuzzyVariable(
        universe_range=(-5, 25),
        terms={
            "nicht attraktiv": ('trapmf', -5, -5, 2, 6),
            "attraktiv": ('trapmf', 2, 6, 11, 15),
            "sehr attraktiv": ('trapmf', 11, 15, 25, 25),
        }
    ),
    "prob": FuzzyVariable(
        universe_range=(0, 50),
        terms={
            "nahe 0": ('trapmf', 0, 0, 3, 7),
            "gering": ('trapmf', 3, 7, 16, 20),
            "hoch": ('trapmf', 16, 20, 36, 40),
            "sehr hoch": ('trapmf', 36, 40, 50, 50),
        }
    )
}

#### Definition der Fuzzy-Regelbasis 

Die gelisteten Regeln stammen aus der Entscheidungsmatrix

In [ ]:
def add_Rule(age, time, P_input_kat, P_output_kat, cons, input_kat, output_kat):
    return FuzzyRule(
        premise=[
            ("age",age),("AND","time",time),
            ("AND","P_kat"+str(input_kat),P_input_kat),
            ("AND","P_kat"+str(output_kat),P_output_kat),],
        consequence=[("prob", cons)]
    )

In [ ]:
ruleBase_1_to_2 = [
    add_Rule("jung", "kurz", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "kurz", "nicht attraktiv", "attraktiv", "hoch", 1, 2),
    add_Rule("jung", "kurz", "nicht attraktiv", "sehr attraktiv", "sehr hoch", 1, 2),
    
    add_Rule("jung", "kurz", "attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "kurz", "attraktiv", "attraktiv", "hoch", 1, 2),
    add_Rule("jung", "kurz", "attraktiv", "sehr attraktiv", "sehr hoch", 1, 2),
    
    add_Rule("jung", "kurz", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "kurz", "sehr attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "kurz", "sehr attraktiv", "sehr attraktiv", "gering", 1, 2),

    
    add_Rule("jung", "mittel", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "mittel", "nicht attraktiv", "attraktiv", "hoch", 1, 2),
    add_Rule("jung", "mittel", "nicht attraktiv", "sehr attraktiv", "sehr hoch", 1, 2),
    
    add_Rule("jung", "mittel", "attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "mittel", "attraktiv", "attraktiv", "gering", 1, 2),
    add_Rule("jung", "mittel", "attraktiv", "sehr attraktiv", "hoch", 1, 2),
    
    add_Rule("jung", "mittel", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "mittel", "sehr attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "mittel", "sehr attraktiv", "sehr attraktiv", "gering", 1, 2),  


    add_Rule("jung", "lang", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "lang", "nicht attraktiv", "attraktiv", "gering", 1, 2),
    add_Rule("jung", "lang", "nicht attraktiv", "sehr attraktiv", "sehr hoch", 1, 2),
    
    add_Rule("jung", "lang", "attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "lang", "attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "lang", "attraktiv", "sehr attraktiv", "hoch", 1, 2),
    
    add_Rule("jung", "lang", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "lang", "sehr attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("jung", "lang", "sehr attraktiv", "sehr attraktiv", "gering", 1, 2),     


    add_Rule("mittel", "kurz", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "kurz", "nicht attraktiv", "attraktiv", "hoch", 1, 2),
    add_Rule("mittel", "kurz", "nicht attraktiv", "sehr attraktiv", "hoch", 1, 2),
    
    add_Rule("mittel", "kurz", "attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "kurz", "attraktiv", "attraktiv", "gering", 1, 2),
    add_Rule("mittel", "kurz", "attraktiv", "sehr attraktiv", "hoch", 1, 2),
    
    add_Rule("mittel", "kurz", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "kurz", "sehr attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "kurz", "sehr attraktiv", "sehr attraktiv", "gering", 1, 2),  


    add_Rule("mittel", "mittel", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "mittel", "nicht attraktiv", "attraktiv", "gering", 1, 2),
    add_Rule("mittel", "mittel", "nicht attraktiv", "sehr attraktiv", "hoch", 1, 2),
    
    add_Rule("mittel", "mittel", "attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "mittel", "attraktiv", "attraktiv", "gering", 1, 2),
    add_Rule("mittel", "mittel", "attraktiv", "sehr attraktiv", "hoch", 1, 2),
    
    add_Rule("mittel", "mittel", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "mittel", "sehr attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "mittel", "sehr attraktiv", "sehr attraktiv", "gering", 1, 2),


    add_Rule("mittel", "lang", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "lang", "nicht attraktiv", "attraktiv", "gering", 1, 2),
    add_Rule("mittel", "lang", "nicht attraktiv", "sehr attraktiv", "hoch", 1, 2),
    
    add_Rule("mittel", "lang", "attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "lang", "attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "lang", "attraktiv", "sehr attraktiv", "hoch", 1, 2),
    
    add_Rule("mittel", "lang", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "lang", "sehr attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("mittel", "lang", "sehr attraktiv", "sehr attraktiv", "nahe 0", 1, 2),    


    add_Rule("alt", "kurz", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "kurz", "nicht attraktiv", "attraktiv", "gering", 1, 2),
    add_Rule("alt", "kurz", "nicht attraktiv", "sehr attraktiv", "hoch", 1, 2),
    
    add_Rule("alt", "kurz", "attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "kurz", "attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "kurz", "attraktiv", "sehr attraktiv", "gering", 1, 2),
    
    add_Rule("alt", "kurz", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "kurz", "sehr attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "kurz", "sehr attraktiv", "sehr attraktiv", "nahe 0", 1, 2),


    add_Rule("alt", "mittel", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "mittel", "nicht attraktiv", "attraktiv", "gering", 1, 2),
    add_Rule("alt", "mittel", "nicht attraktiv", "sehr attraktiv", "gering", 1, 2),
    
    add_Rule("alt", "mittel", "attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "mittel", "attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "mittel", "attraktiv", "sehr attraktiv", "gering", 1, 2),
    
    add_Rule("alt", "mittel", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "mittel", "sehr attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "mittel", "sehr attraktiv", "sehr attraktiv", "nahe 0", 1, 2),


    add_Rule("alt", "lang", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "lang", "nicht attraktiv", "attraktiv", "gering", 1, 2),
    add_Rule("alt", "lang", "nicht attraktiv", "sehr attraktiv", "gering", 1, 2),
    
    add_Rule("alt", "lang", "attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "lang", "attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "lang", "attraktiv", "sehr attraktiv", "gering", 1, 2),
    
    add_Rule("alt", "lang", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "lang", "sehr attraktiv", "attraktiv", "nahe 0", 1, 2),
    add_Rule("alt", "lang", "sehr attraktiv", "sehr attraktiv", "nahe 0", 1, 2),
]

#print(rules_change12)

In [ ]:
ruleBase_1_to_3 = [
    add_Rule("jung", "kurz", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "kurz", "nicht attraktiv", "attraktiv", "hoch", 1, 3),
    add_Rule("jung", "kurz", "nicht attraktiv", "sehr attraktiv", "sehr hoch", 1, 3),
    
    add_Rule("jung", "kurz", "attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "kurz", "attraktiv", "attraktiv", "gering", 1, 3),
    add_Rule("jung", "kurz", "attraktiv", "sehr attraktiv", "sehr hoch", 1, 3),
    
    add_Rule("jung", "kurz", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "kurz", "sehr attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "kurz", "sehr attraktiv", "sehr attraktiv", "gering", 1, 3),
    
    #-------------------------

    add_Rule("jung", "mittel", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "mittel", "nicht attraktiv", "attraktiv", "hoch", 1, 3),
    add_Rule("jung", "mittel", "nicht attraktiv", "sehr attraktiv", "sehr hoch", 1, 3),
    
    add_Rule("jung", "mittel", "attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "mittel", "attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "mittel", "attraktiv", "sehr attraktiv", "hoch", 1, 3),
    
    add_Rule("jung", "mittel", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "mittel", "sehr attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "mittel", "sehr attraktiv", "sehr attraktiv", "gering", 1, 3),  

    #--------------

    add_Rule("jung", "lang", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "lang", "nicht attraktiv", "attraktiv", "gering", 1, 3),
    add_Rule("jung", "lang", "nicht attraktiv", "sehr attraktiv", "hoch", 1, 3),
    
    add_Rule("jung", "lang", "attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "lang", "attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "lang", "attraktiv", "sehr attraktiv", "gering", 1, 3),
    
    add_Rule("jung", "lang", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "lang", "sehr attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("jung", "lang", "sehr attraktiv", "sehr attraktiv", "nahe 0", 1, 3),  

    #----------

    add_Rule("mittel", "kurz", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "kurz", "nicht attraktiv", "attraktiv", "hoch", 1, 3),
    add_Rule("mittel", "kurz", "nicht attraktiv", "sehr attraktiv", "hoch", 1, 3),
    
    add_Rule("mittel", "kurz", "attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "kurz", "attraktiv", "attraktiv", "gering", 1, 3),
    add_Rule("mittel", "kurz", "attraktiv", "sehr attraktiv", "hoch", 1, 3),
    
    add_Rule("mittel", "kurz", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "kurz", "sehr attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "kurz", "sehr attraktiv", "sehr attraktiv", "nahe 0", 1, 3),  

    #-------

    add_Rule("mittel", "mittel", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "mittel", "nicht attraktiv", "attraktiv", "gering", 1, 3),
    add_Rule("mittel", "mittel", "nicht attraktiv", "sehr attraktiv", "hoch", 1, 3),
    
    add_Rule("mittel", "mittel", "attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "mittel", "attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "mittel", "attraktiv", "sehr attraktiv", "gering", 1, 3),
    
    add_Rule("mittel", "mittel", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "mittel", "sehr attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "mittel", "sehr attraktiv", "sehr attraktiv", "nahe 0", 1, 3),

    #---------

    add_Rule("mittel", "lang", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "lang", "nicht attraktiv", "attraktiv", "gering", 1, 3),
    add_Rule("mittel", "lang", "nicht attraktiv", "sehr attraktiv", "hoch", 1, 3),
    
    add_Rule("mittel", "lang", "attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "lang", "attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "lang", "attraktiv", "sehr attraktiv", "gering", 1, 3),
    
    add_Rule("mittel", "lang", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "lang", "sehr attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("mittel", "lang", "sehr attraktiv", "sehr attraktiv", "nahe 0", 1, 3),   

    #--------

    add_Rule("alt", "kurz", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "kurz", "nicht attraktiv", "attraktiv", "gering", 1, 3),
    add_Rule("alt", "kurz", "nicht attraktiv", "sehr attraktiv", "hoch", 1, 3),
    
    add_Rule("alt", "kurz", "attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "kurz", "attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "kurz", "attraktiv", "sehr attraktiv", "gering", 1, 3),
    
    add_Rule("alt", "kurz", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "kurz", "sehr attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "kurz", "sehr attraktiv", "sehr attraktiv", "nahe 0", 1, 3),

    #--------

    add_Rule("alt", "mittel", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "mittel", "nicht attraktiv", "attraktiv", "gering", 1, 3),
    add_Rule("alt", "mittel", "nicht attraktiv", "sehr attraktiv", "gering", 1, 3),
    
    add_Rule("alt", "mittel", "attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "mittel", "attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "mittel", "attraktiv", "sehr attraktiv", "gering", 1, 3),
    
    add_Rule("alt", "mittel", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "mittel", "sehr attraktiv", "attraktiv", "gering", 1, 3),
    add_Rule("alt", "mittel", "sehr attraktiv", "sehr attraktiv", "gering", 1, 3),
    
    #-----

    add_Rule("alt", "lang", "nicht attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "lang", "nicht attraktiv", "attraktiv", "gering", 1, 3),
    add_Rule("alt", "lang", "nicht attraktiv", "sehr attraktiv", "gering", 1, 3),
    
    add_Rule("alt", "lang", "attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "lang", "attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "lang", "attraktiv", "sehr attraktiv", "gering", 1, 3),
    
    add_Rule("alt", "lang", "sehr attraktiv", "nicht attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "lang", "sehr attraktiv", "attraktiv", "nahe 0", 1, 3),
    add_Rule("alt", "lang", "sehr attraktiv", "sehr attraktiv", "nahe 0", 1, 3),    
]

#### Anwendung des Fuzzy-Inferenz-Schemas mit Defuzzifizierung

In [ ]:
model = DecompositionalInference(
    and_operator=and_operator,
    or_operator=or_operator,
    implication_operator=implication_operator,
    composition_operator="max-min", #sollte so bleiben
    production_link=or_operator, #sollte gleich or_operator sein
    defuzzification_operator=defuzzification_operator,
)

print(round(list(model(
    variables=variables,
    rules=ruleBase_1_to_2,
    age=age,
    time=time,
    P_kat1=P_kat1,
    P_kat2=P_kat2,
)[0].items())[0][1],2))

print(round(list(model(
    variables=variables,
    rules=ruleBase_1_to_3,
    age=age,
    time=time,
    P_kat1=P_kat1,
    P_kat3=P_kat3,
)[0].items())[0][1],2))